In [ ]:
import requests
from tqdm import tqdm

import csv
import datetime
import pickle

In [ ]:
sources = {
    "cases": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
        },
    "deaths": {
        "url": "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
        }
    }

In [ ]:
firstdate = "1/22/20"

In [ ]:
masterdict = {}
statedict = {}
datetranslate = {}

In [ ]:
for source in sources:
    url = sources[source]['url']
    r = requests.get(url)
    localdata = r.text.splitlines()
    
    # Identify dates and build a table
    if source == "cases":
        headers = localdata[0].split(",")
        datestart = headers.index(firstdate)
        for localdate in headers[datestart:]:
            gooddate = datetime.datetime.strptime(localdate, "%m/%d/%y").strftime("%Y%m%d")
            datetranslate[localdate] = gooddate

    # Start parsing the CSV as a CSV
    parsedcsv = csv.DictReader(localdata)
    for row in tqdm(parsedcsv):
        fips = row['FIPS']
        if len(fips) == 0:
            fips = row['Combined_Key']
        elif ".0" in fips:
            fips = str(int(fips.replace(".0", ""))).zfill(5)
        county = row['Admin2']
        state = row['Province_State']
        if source == "deaths":
            pop = int(row['Population'])
        if fips not in masterdict:
            masterdict[fips] = {}
        for baddate in datetranslate:
            gooddate = datetranslate[baddate]
            if gooddate not in masterdict[fips]:
                masterdict[fips][gooddate] = {}
                masterdict[fips][gooddate]["fips"] = fips
                masterdict[fips][gooddate]["state"] = state
                masterdict[fips][gooddate]["county"] = county
                masterdict[fips][gooddate]["pop"] = None
                masterdict[fips][gooddate]["mydate"] = gooddate
            if source == "deaths":
                masterdict[fips][gooddate]["pop"] = pop
            localvalue = int(row[baddate])
            masterdict[fips][gooddate][source] = localvalue

In [ ]:
masterdict['12099']['20230101']

In [ ]:
# Build a straight list of valid dates in yyymmdd formats
mydates = []
for baddate in datetranslate:
    mydates.append(datetranslate[baddate])

In [ ]:
for i, mydate in enumerate(mydates):
    for fips in masterdict:
        
        # First pass, build out per-day values
        for source in sources:
            short = source[0]     # Use c(ases) and d(eaths) as abbreiations
            if i == 0:
                masterdict[fips][mydate][short + "day"] = masterdict[fips][mydate][source]
            else:
                diff = masterdict[fips][mydate][source] - masterdict[fips][mydates[i-1]][source]
                masterdict[fips][mydate][short + "day"] = diff
                
        # Second pass, build out per-week values
        for source in sources:
            short = source[0]     # Use c(ases) and d(eaths) as abbreiations
            if i < 7:
                masterdict[fips][mydate][short + "week"] = masterdict[fips][mydate][source]
            else:
                diff = masterdict[fips][mydate][source] - masterdict[fips][mydates[i-7]][source]
                masterdict[fips][mydate][short + "week"] = diff

In [ ]:
with open("full-report.csv", "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(list(masterdict['12099'][mydates[0]].keys()))
    for fips in masterdict:
        for mydate in mydates:
            writer.writerow(list(masterdict[fips][mydate].values()))

In [ ]:
list(masterdict['12099'][mydates[0]].keys())

In [ ]:
statedict = {}
for fips in masterdict:
    for mydate in mydates:
        z = masterdict[fips][mydate]
        state = z['state']
        if state not in statedict:
            statedict[state] = {}
        if mydate not in statedict[state]:
            statedict[state][mydate] = {}
            statedict[state][mydate]["state"] = state
            statedict[state][mydate]["pop"] = 0
            statedict[state][mydate]["mydate"] = mydate
            for item in ["cases", "deaths", "cday", "dday", "cweek", "dweek"]:
                statedict[state][mydate][item] = 0
        for item in ["pop", "cases", "deaths", "cday", "dday", "cweek", "dweek"]:
            statedict[state][mydate][item] += z[item]

In [ ]:
with open("state-report.csv", "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(list(statedict["Florida"]["20220101"].keys()))
    for state in statedict:
        for mydate in mydates:
            writer.writerow(list(statedict[state][mydate].values()))